# Installing Libraries

In [1]:
!pip install datasets pyverilog pyrtl xgboost matplotlib seaborn scikit-learn
!sudo apt-get install yosys

Defaulting to user installation because normal site-packages is not writeable


Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app


In [2]:
from datasets import load_dataset

# Loading the Dataset

In [3]:
dataset = load_dataset('emilgoh/verilog-dataset-v2', split='train')
dataset = dataset.take(1000)

In [4]:
import subprocess
import re
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

# Processing Block
This is done using the YoSys Open Synthesis Suite to extract structural synthesis details for RTL Modules in Dataset to extract features for our Dataset 

In [35]:
import subprocess
import re
import pandas as pd
from tqdm import tqdm

def extract_top_module(rtl_code):
    """Extract the top module name from RTL code."""
    match = re.search(r'module\s+(\w+)', rtl_code)
    return match.group(1) if match else None

def get_logic_depth_1(rtl_code, yosys_path='C:\\yosys\\oss-cad-suite\\bin\\yosys.exe'):
    """Run synthesis and extract features using Yosys."""
    try:
        # Extract top module name from RTL code
        top_module = extract_top_module(rtl_code)
        if not top_module:
            raise ValueError("No module declaration found in RTL code")

        # Write RTL code to a temporary file
        with open('temp_design.v', 'w', encoding='utf-8') as f:
            f.write(rtl_code)

        # Command to run environment.bat and Yosys
        command = (
            f'C:\\yosys\\oss-cad-suite\\environment.bat && '
            f'{yosys_path} -p "read_verilog temp_design.v; synth -top {top_module}; stat"'
        )

        # Run the command
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            timeout=30,
            shell=True
        )

        # Debugging
        #print("Yosys Output:", result.stdout)

        # Extract features from Yosys output
        features = {
            'num_wires': int(re.search(r'Number of wires:\s+(\d+)', result.stdout).group(1)) 
                        if re.search(r'Number of wires:\s+(\d+)', result.stdout) else 0,
            'num_cells': int(re.search(r'Number of cells:\s+(\d+)', result.stdout).group(1)) 
                        if re.search(r'Number of cells:\s+(\d+)', result.stdout) else 0,
            'num_memories': int(re.search(r'Number of memories:\s+(\d+)', result.stdout).group(1)) 
                          if re.search(r'Number of memories:\s+(\d+)', result.stdout) else 0,
            'has_warnings': "Warning" in result.stdout,
            'has_errors': "ERROR" in result.stdout,
            'RTL Code' : rtl_code
        }
        

       

        return features

   
    except Exception as e:
        print(f"Synthesis failed: {e}")
        return {}


def create_dataset_1(dataset, sample_size=200, yosys_path='C:\\yosys\\oss-cad-suite\\bin\\yosys.exe'):
    """Create feature-label dataset from RTL samples (no sanitization)."""
    data = []
    for i, sample in enumerate(tqdm(dataset, desc="Processing")):
        if i >= sample_size:
            break

        # Extract raw RTL code without modification
        rtl_code = sample.get('output', '')
        if not rtl_code:
            print(f"Sample {i}: Empty RTL code")
            continue

        try:
            # Extract features directly from raw RTL code
            features = get_logic_depth_1(rtl_code, yosys_path=yosys_path)
            
            # Only append if features are valid
            if features.get('num_cells', 0) > 0: 
                data.append({
                    **features,
                    'num_lines': len(rtl_code.split('\n')),
                    'sample_id': i
                })
            else:
                print(f"Sample {i}: Invalid features (num_cells=0)")

        except Exception as e:
            print(f"Sample {i}: Error - {str(e)}")

    return pd.DataFrame(data)

# Creating the required Dataset
We are analyzing first 1000 samples of the given dataset and extracting details for the RTL Modules which can be synthesized to create the dataset we require

In [30]:
df=create_dataset_1(dataset,1000)
df.to_csv('rtldataset.csv',index=False)

Processing:   0%|▏                                                                    | 3/1000 [00:00<03:11,  5.21it/s]

Sample 1: Invalid features (num_cells=0)
Sample 2: Invalid features (num_cells=0)


Processing:   0%|▎                                                                    | 4/1000 [00:00<03:06,  5.33it/s]

Sample 3: Invalid features (num_cells=0)


Processing:   1%|▉                                                                   | 14/1000 [00:03<03:51,  4.27it/s]

...

Sample 990: Invalid features (num_cells=0)
Sample 991: Invalid features (num_cells=0)
Sample 992: Invalid features (num_cells=0)


Processing: 100%|██████████████████████████████████████████████████████████████████▋| 996/1000 [03:07<00:00,  9.91it/s]

Sample 993: Invalid features (num_cells=0)
Sample 994: Invalid features (num_cells=0)
Sample 995: Invalid features (num_cells=0)
Sample 996: Invalid features (num_cells=0)


Processing: 100%|██████████████████████████████████████████████████████████████████| 1000/1000 [03:08<00:00,  5.30it/s]

Sample 998: Invalid features (num_cells=0)
Sample 999: Invalid features (num_cells=0)


# Dataset overview

In [37]:
df.head()

,num_wires,num_cells,num_memories,has_warnings,has_errors,RTL Code,num_lines,sample_id
0,317,434,0,True,False,module axi_protocol_converter_v2_1_b2s_simple_...,73,0
1,252,365,0,False,False,module altera_avalon_sc_fifo\n#(\n // -----...,895,4
2,32,8,0,False,False,module axi_protocol_converter_v2_1_r_axi3_conv...,153,5
3,34,64,0,False,False,"module e0 (x, y);\n\n\tinput [31:0] x;\n\toutp...",8,6
4,34,64,0,False,False,"module e1 (x, y);\n\n\tinput [31:0] x;\n\toutp...",8,7


Example input to test how structural synthesis is extracted

In [33]:

rt1="""
module jk_flipflop(
    input j,
    input k,
    input clk,
    output q,
    output qbar
    );
    reg q,qbar;
    always @(negedge clk)
        begin
        case({j,k})
        2'b00 : q <= q;
        2'b01 : q <= 0;
        2'b10 : q <= 1;
        2'b11 : q <= ~q;
        endcase
    qbar = ~q;
    end
     
endmodule
"""
rtl_code = """
module adder(input [3:0] a, b, output [4:0] sum);
    assign sum = a + b;
endmodule
"""

print(get_logic_depth_1(rtl_code))

{'num_wires': 18, 'num_cells': 20, 'num_memories': 0, 'has_warnings': False, 'has_errors': False, 'RTL Code': '\nmodule adder(input [3:0] a, b, output [4:0] sum);\n    assign sum = a + b;\nendmodule\n'}
